Initialization

In [ ]:
import math
import json
import os
import random
from IPython import display
import sympy as sp
import numpy as np
import pandas as pd
import sklearn


GREEN = "\033[92m"
RED = "\033[91m"
YELLOW = "\033[93m"
BLUE = "\033[94m"
CYAN = "\033[96m"
MAGENTA = "\033[95m"
BOLD = "\033[1m"
UNDERLINE = "\033[4m"
STANDARD = "\033[0m"

%run config.py

In [ ]:
def csv2sequence(csvFile,seqFile,pref):
	with open(csvFile, 'r') as f:
		lines = f.readlines()
		with open(seqFile, 'w') as g:
			for line in lines:
				for token in line.split(','):
					if token.startswith(pref):
						token = token[len(pref):]
					token = token.strip()
					if token!="" and token!='\n':
						g.write(token+'\n')

def sequence2csv(seqFile,csvFile,dataWidth,pref):
	with open(seqFile, 'r') as f:
		lines = f.readlines()
		dataW = dataWidth
		with open(csvFile, 'w') as g:
			for line in lines:
				g.write(pref+line.strip())
				dataW -= 1
				if dataW==0:
					g.write('\n')
					dataW = dataWidth
				else:
					g.write(',')

Create the expression using flexpytester and create inputs and outputs for the test cases.

In [ ]:
def generate():
	! flexpytester --generate -e symbols -s generated.py -i inputs.csv -o outputs.csv --prefix --config decayFactor=3 > /dev/null
	! bmhelper apply 
	! make clean > /dev/null
	! make show > /dev/null
	csv2sequence('inputs.csv','inputs.seq',"0f")
	csv2sequence('outputs.csv','outputs.seq',"0f")
	# sequence2csv('inputs.seq','prova',4,"0f")
# generate()
# display.Image("bondmachine.png")

In [ ]:
# exec(open("generated.py").read())
# import IPython.display as disp
# disp.display(spExpr)

Eventually run the simbatch simulation

In [ ]:
def simbatch():
	! make simbatch > /dev/null
# simbatch()

Eventually run the simulation in the FPGA with the BM

In [ ]:
def bmsim():
	! make deploycollect > /dev/null
	! mv working_dir/bmsim_outputs.seq bmsim_outputs.seq
	! bondmachine -bondmachine-file working_dir/bondmachine.json -list-outputs | wc -l > num_outputs.txt
	# Read the number of outputs
	with open('num_outputs.txt','r') as f:
		num_outputs = int(f.read())
	sequence2csv('bmsim_outputs.seq','bmsim_outputs.csv',num_outputs,"")

Eventually run the simulation in the FPGA with the HLS technology

In [ ]:
def hlsim():
	print ("hlsim")

Analyze the results of the chosen cases

In [ ]:
def analyze():
	# Load the target outputs from the output.csv file
	targetData = np.loadtxt('outputs.csv', delimiter=',')
	simbatchData = np.loadtxt('simbatch_outputs.csv', delimiter=',')
	# bmsimData = np.loadtxt('bmsim_outputs.csv', delimiter=',')
	simbatchMSE=sklearn.metrics.mean_squared_error(targetData, simbatchData)
	# bmsimMSE=sklearn.metrics.mean_squared_error(targetData, bmsimData)

	data = {
		# "Dataset": ["Sympy target", "Simbatch", "BondMachine"],
		# "MSE" : [0.0, simbatchMSE, bmsimMSE]
		"Dataset": ["Sympy target", "Simbatch"],
		"MSE" : [0.0, simbatchMSE]
	}
	df = pd.DataFrame(data)
	return df.style.hide(axis="index"), simbatchMSE

	

Main test loop

In [ ]:
# for i in range(numiterations):
# print ("Iteration: ", i)
generate()
simbatch()
# bmsim()
# hlsim()
errors,mse=analyze()
errors

In [ ]:
display.Image("bondmachine.png")

In [ ]:
exec(open("generated.py").read())
import IPython.display as disp
disp.display(spExpr)

In [ ]:
# Load the statistics from the JSON file as draw an histogram
with open('statistics.json', 'r') as f:
    data = json.load(f)

df = pd.DataFrame.from_dict(data, orient='index', columns=['occurrences'])
totOccurrences = df['occurrences'].sum()
newCol=pd.DataFrame(df['occurrences'])
newCol=newCol * mse / totOccurrences
df['error'] = newCol
df

In [ ]:
# Plot the histogram
df['occurrences'].plot(kind='bar', figsize=(10, 6), color='skyblue')


In [ ]:
# Load the global statistics from the CSV file
global_stats = pd.read_csv('global_statistics.csv', index_col=0)
global_stats

In [ ]:
# Sum the occurrences to the global statistics
global_stats += df
# Save the updated global statistics
global_stats.to_csv('global_statistics.csv')

In [ ]:
# Plot the histogram of the global statistics
global_stats['occurrences'].plot(kind='bar', figsize=(10, 6), color='skyblue')